In [1]:
from kalah import *
# from pebblereaper import *
from extrakalah import *
depth = 3
weights = [0.51, 0.01, 0.34, 0.1]
h_list = [pit_total,side_total,chance_to_clear_right_most,congregate_seeds]
full_h_list = [pit_total,side_total,chance_to_clear_right_most,congregate_seeds,capture_defense]
alt_h_list = [pit_total,side_total,chance_to_clear_right_most,congregate_seeds,capture_available]

class Jogador():
    def __init__(self, nome, fun):
        self.nome = nome
        self.fun = fun
    def display(self):
        print(self.nome+" ")
        
class JogadorAlfaBeta(Jogador):
    def __init__(self, nome, depth,fun_eval):
        self.nome = nome
        self.fun = lambda game, state: alphabeta_cutoff_search_new(state,game,depth,eval_fn=fun_eval)


##########  para ser independente dos jogos deveria devolver um método em string ou um atributo
def joga11(game, jog1, jog2,verbose=False):
    ### jog1 e jog2 são jogadores com funções que dado um estado do jogo devolvem a jogada que escolheram
    ### devolve o par de jogadores, a lista de jogadas e o resultado
    estado=game.initial
    proxjog = jog1
    lista_jogadas=[]
    lance = 0
    while not game.terminal_test(estado):
        if verbose:
            print('----------   LANCE:',lance)
            game.display(estado)
        jogada = proxjog.fun(game, estado) # state produced
        if verbose:
            print('JOGADA=',jogada)
            print("pit_total:",pit_total(estado,proxjog))
            print("side_total: ", side_total(estado,proxjog))
            print("chance_to_clear_right_most:",chance_to_clear_right_most(estado,proxjog))
            print("congregate_seeds: ", congregate_seeds(estado,proxjog))
            print("heuristic total:",linearPond(estado,proxjog,weights,h_list,100))
        estado=game.result(estado,jogada)
        lista_jogadas.append(jogada)
        proxjog = jog2 if proxjog == jog1 else jog1
        lance+=1
    #p jogou e ganhou
    util=game.utility(estado,0)
    if util == 1:
        resultado=jog1.nome
    elif util== -1:
        resultado = jog2.nome
    else:
        resultado='Empate'
    return ((jog1.nome,jog2.nome),lista_jogadas,resultado)


100%|██████████| 10000/10000 [00:00<00:00, 1930101.70it/s]

## Create Chapiteau

In [2]:
def toes(state,player):
    board = state.state
    if state.is_game_over():
        aux = state.result()
        return aux*100 if player == state.SOUTH else aux*-100 
    
    own_seeds = sum(board[:7]) if player == 0 else sum(board[7:14])
    opponent_seeds = sum(board[7:14]) if player == 0 else sum(board[:7])
    score = own_seeds - opponent_seeds
    return score

toe1 = JogadorAlfaBeta("chapiteau",depth,toes)
# toe2 = JogadorAlfaBeta("heehoo",6,toes)



## Create Random Player

In [3]:
def f_caos_intel(estado,jogador):
    """Quando é terminal: +100 para vitória, -100 para a derrota e 0 para o empate.
       Quando o tabuleiro é não terminal devolve 0, o que quer dizer que como o minimax baralha as acções, será random"""
    if estado.is_game_over():
        aux = estado.result()
        return aux*100 if jogador == estado.SOUTH else aux*-100
    return 0

el_caos_int6=JogadorAlfaBeta("El Caos Inteligente 6",depth,f_caos_intel)

# Pebbles!

In [4]:
import copy
# pebs = JogadorAlfaBeta('Pebbles',depth,Jogador_42)
# nome, depth,weights,features,win_value
h_list = [pit_total,side_total,chance_to_clear_right_most,congregate_seeds]
pebs =JogadorLinearPond('Pebbles',depth,weights,h_list,100)

#
# defense focused
#

# pebs2: more ags trained against chapiteau
# weights2 = copy.copy(weights)
# weights2.append(1)
weights2 = [0.74, 0.01, 0.09, 0.06, 0.15]
pebs2 = JogadorLinearPond('Pebbles2-defense-focused',depth,weights2,full_h_list,100)

# depth  8 gen 7
# weights3 = [0.368, 0.0, 0.242,0.152,0.237] #[0.74, 0.01, 0.09, 0.06, 0.15]
# pebs3 = JogadorLinearPond('Pebbles3-defense-focused',depth,weights3,full_h_list,100)

weights4 =  [0.53, 0.01, 0.03, 0.03, 0.04]
pebs4 = JogadorLinearPond('Pebbles4-defense-focused',depth,weights4,full_h_list,100)

#
# capture focused
#
weights2b = [0.74, 0.01, 0.09, 0.06, 0.15]
pebs2b = JogadorLinearPond('Pebbles2-capture-focused',depth,weights2b,alt_h_list,100)

# weights3b = [0.368, 0.0, 0.242,0.152,0.237] #[0.74, 0.01, 0.09, 0.06, 0.15]
# pebs3b = JogadorLinearPond('Pebbles3-capture-focused',depth,weights3b,alt_h_list,100)

weights4b =  [0.53, 0.01, 0.03, 0.03, 0.04]
pebs4b = JogadorLinearPond('Pebbles4-capture-focused',depth,weights4b,alt_h_list,100)


# One game

In [5]:
mx = Kalah(0)
_,_,res=joga11(jogo,pebs2,pebs,True)
res

----------   LANCE: 0
----------------------
     _  _  _  _  _  _
     1  2  3  4  5  6
-------------------------
|  | 4| 4| 4| 3| 4| 4|  |
| 2|-----------------| 0|
|  | 3| 4| 4| 4| 4| 4|  | <--
-------------------------
     6  5  4  3  2  1

JOGADA= 3
pit_total: 2
side_total:  0
chance_to_clear_right_most: 0
congregate_seeds:  0.7999999999999998
heuristic total: 1.1
----------   LANCE: 1
----------------------
     _  _  _  _  _  _
     1  2  3  4  5  6
-------------------------
|  | 4| 4| 4| 3| 4| 5|  | <--
| 2|-----------------| 1|
|  | 3| 4| 4| 0| 5| 5|  |
-------------------------
     6  5  4  3  2  1

JOGADA= 12
pit_total: 1
side_total:  3
chance_to_clear_right_most: 0
congregate_seeds:  1.7999999999999998
heuristic total: 0.72
----------   LANCE: 2
----------------------
     _  _  _  _  _  _
     1  2  3  4  5  6
-------------------------
|  | 0| 4| 4| 3| 4| 5|  |
| 3|-----------------| 1|
|  | 4| 5| 5| 0| 5| 5|  | <--
-------------------------
     6  5  4  3  2  1

JOGADA

'Pebbles2-defense-focused'

# N Match Pairs

In [6]:
scores={'Vitoria': 3, 'Empate': 1}

def traduzPontos(tabela):
    tabelaScore={}
    empates=tabela['Empate']
    for x in tabela:
        if x != 'Empate':
            tabelaScore[x]=scores['Vitoria']*tabela[x]+empates
    return tabelaScore

def getPoints(tabela, jogador):
    empates=tabela['Empate']
    total=0
    for x in tabela:
        total += scores['Vitoria']*tabela[x]+empates
    player_score = scores['Vitoria']*tabela[jogador]+empates
    return player_score/total

#ret_jog_index == 0 or 1 ONLY
def jogaNpares_GA(jogo,n,jog1,jog2,ret_jog_index):
    tabelaPrim={jog1.nome:0, jog2.nome:0, 'Empate':0}
    tabelaSeg={jog1.nome:0, jog2.nome:0, 'Empate':0}
    tabela={}
    print(' ', end='', flush=True)
    for _ in tqdm(range(n), desc= "jogaNpares_GA range %d"%jog2.nome,leave=False):
        _,_,vencedor=joga11(jogo,jog1,jog2)
        tabelaPrim[vencedor]+=1
        _,_,vencedor=joga11(jogo,jog2,jog1)
        tabelaSeg[vencedor]+=1
    for x in tabelaPrim:
        tabela[x]=tabelaPrim[x]+tabelaSeg[x]
    ret_jog = jog1.nome if ret_jog_index == 0 else jog2.nome
    return getPoints(tabela,ret_jog) # this is needed for GA

def jogaNpares(jogo,n,jog1,jog2):
    tabelaPrim={jog1.nome:0, jog2.nome:0, 'Empate':0}
    tabelaSeg={jog1.nome:0, jog2.nome:0, 'Empate':0}
    tabela={}
    for _ in range(n):
        _,_,vencedor=joga11(jogo,jog1,jog2)
        tabelaPrim[vencedor]+=1
        _,_,vencedor=joga11(jogo,jog2,jog1)
        tabelaSeg[vencedor]+=1
    for x in tabelaPrim:
        tabela[x]=tabelaPrim[x]+tabelaSeg[x]
    # ret_jog = jog1.nome if ret_jog_index == 0 else jog2.nome
    # return getPoients(tabela,ret_jog) # this is needed for GA
    return tabelaPrim,tabelaSeg,tabela,traduzPontos(tabela) # this gives better data for individual npare runs

# SUPER IMPORTANT: if you want to see the full table using jogaNpares rather than the numerical win rate for GA, comment/uncomment the last two lines above

In [7]:
# jogaNpares()

jogo=Kalah(0)
#n jogaNpares(jogo,300,el_caos,el_caos_int6)
# jogo.result()

# _,_,res=joga11(jogo,pebs,pebs2,True)
# jogaNpares(jogo,30,toe1,pebs)
# jogaNpares(jogo,30,toe1,pebs2)
# jogaNpares(jogo,30,toe1,pebs4)
# print(res)


# SearchPlus Genetic Algorithm

In [8]:
# from searchPlus import genetic_algorithm_custom
# import random

# # for steps in 0.01, use 2. 
# DEPTH = depth 
# PRECISION = 10
# HEURISTICS = alt_h_list
# N_WEIGHTS = 5
# TRAINING_ENEMY = pebs4
# N_GAMES = 10
# N_GENERATION = 1
# PMUT = 0.1
# # generates n weights that adds up to 1, precision see above
# # e.g. [0.7, 0.12, 0.12, 0.06]
# def generate_population(n,precision):
#     total = pow(10,precision)
#     dividers = random.sample(range(1, total), n - 1)
#     return [(a - b)/total for a, b in zip(dividers + [total], [0] + dividers)]

# # generates a pool of potential changes from 0-1
# # a list that goes [0.0, 0.01 ... 1] essentially. based on precision
# def generate_pool(precision):
#     return [round(x*pow(10,-precision),precision) for x in range(pow(10,precision))]
    
# # generates 8 "gene" as an initial population
# pop = [generate_population(N_WEIGHTS,PRECISION) for _ in range(8)]
# pop_pool = generate_pool(PRECISION)
# print(pop)
# print(pop_pool)

# # the fitness function evaluates how well a gene does. in this case we insert the weights
# # into a player into npare games, which i've changed it to return the win percentage
# pebs_generator = lambda name,w:JogadorLinearPond(name,DEPTH,w,HEURISTICS,100)
# fitness= lambda name, w:jogaNpares_GA(jogo,N_GAMES,TRAINING_ENEMY,pebs_generator(name,w),1)

#call the genetic algorithm
# best_fit_weights = genetic_algorithm_custom(pop,fitness,pop_pool,ngen=N_GENERATION,pmut=PMUT)

# print(best_fit_weights)

# KEVDEV GA

In [9]:
#backup - kev dev
from searchPlus import genetic_algorithm_custom
from kevdiff import genetic_algorithm_custom as ga_kev
import random

# for steps in 0.01, use 2. 
DEPTH = depth  #6 rn
# PRECISION = 2
HEURISTICS = alt_h_list
N_WEIGHTS = 5
TRAINING_ENEMY = pebs4
N_GAMES = 50
N_GENERATION = 1
PMUT = 0.1
N_POPULATION = 24
# generates n weights that adds up to 1, precision see above
# e.g. [0.7, 0.12, 0.12, 0.06]
def generate_population(n):
    return normalize([random.random() for _ in range(n)])

# generates a pool of potential changes from 0-1
# a list that goes [0.0, 0.01 ... 1] essentially. based on precision
# def generate_pool(precision):
#     return [round(x*pow(10,-precision),precision) for x in range(pow(10,precision))]
    
# generates 8 "gene" as an initial population
# pop = [generate_population(N_WEIGHTS) for _ in range(N_POPULATION)]

pop_gen17 = [(296, [0.5408094183539636, 0.00284445612406821, 0.15969595598418704, 0.22260996169213218, 0.07404020784564898], 0.5922330097087378), (181, [0.5060064283744496, 0.0023946325203097678, 0.27945500045902055, 0.17827186795515407, 0.03387207069106592], 0.5873015873015873), (326, [0.5087305384943449, 0.0030431687499736727, 0.17085225489545827, 0.2381614091784416, 0.07921262868178149], 0.5849056603773585), (281, [0.5492302031235177, 0.002888746315759254, 0.15673364697615585, 0.2184806179605766, 0.07266678562399057], 0.5754716981132075), (247, [0.41752397451100454, 0.0012991470449279108, 0.29854278707734244, 0.21209212302177424, 0.07054196834495088], 0.573208722741433), (340, [0.4293397173643812, 0.0012727933638494399, 0.2924867355859959, 0.20778975540973707, 0.06911099827603649], 0.5728155339805825), (218, [0.43822936301080534, 0.001363572912484629, 0.31334779174201893, 0.20761242699559582, 0.0394468453390953], 0.5705128205128205), (328, [0.5141678293348594, 0.002536050034560504, 0.16912696677170333, 0.2357564245263881, 0.07841272933248868], 0.5682539682539682), (304, [0.5115214949486129, 0.0011768297898419075, 0.23384833697495405, 0.19019452446170035, 0.06325881382489079], 0.565359477124183), (216, [0.5244964022687261, 0.0024821347540840543, 0.16553140370103708, 0.23074434922059944, 0.07674571005555327], 0.565359477124183), (183, [0.5408094183539636, 0.00284445612406821, 0.15969595598418704, 0.22260996169213218, 0.07404020784564898], 0.565359477124183), (191, [0.4220340143246191, 0.0016217516845741953, 0.3222589517171829, 0.21351662545668781, 0.040568656816935986], 0.5638629283489096), (361, [0.5321287927976377, 0.0024422935796346032, 0.1628744304964742, 0.22704063174297384, 0.0755138513832796], 0.5631067961165048), (244, [0.503236853071467, 0.003219031462752779, 0.2098999632357117, 0.23835596329214934, 0.04528818893791916], 0.5566037735849056), (343, [0.5278617454660057, 0.0012144229189132074, 0.16479734850271316, 0.22972110477729635, 0.07640537833507148], 0.5533980582524272), (286, [0.47551148641798996, 0.0012635841717298889, 0.2510873362804695, 0.204215420729435, 0.06792217240037564], 0.5533980582524272), (337, [0.4824081948782528, 0.00253728743925837, 0.2879892244193175, 0.19081079683976004, 0.036254496423411206], 0.5512820512820513), (119, [0.4567526386179326, 0.0014511037277271798, 0.28834942524079316, 0.04219666311629308, 0.21125016929725385], 0.5471698113207547), (321, [0.4240589846181617, 0.0016295330454913263, 0.3238051891812408, 0.21050907747006667, 0.03999721568503944], 0.545751633986928), (268, [0.4589515295334259, 0.002413914115632853, 0.24539703161033458, 0.2200490707654506, 0.07318845397515614], 0.545751633986928), (145, [0.42096729007497213, 0.0016247448742976692, 0.3228537296919815, 0.21391070290713027, 0.04064353245161829], 0.545751633986928), (205, [0.5242839050426428, 0.0027575380787338167, 0.17980785464343735, 0.21998391808195197, 0.07316678415323409], 0.5451713395638629), (305, [0.4641343575158191, 0.0012333515476795063, 0.24507979898187618, 0.21728377651802178, 0.07226871543660336], 0.5416666666666666), (239, [0.5408094183539636, 0.00284445612406821, 0.15969595598418704, 0.22260996169213218, 0.07404020784564898], 0.5416666666666666)]
pop = []
for ind in pop_gen17:
    pop.append(ind[1])
# pop_pool = generate_pool(PRECISION)
print(pop)
# print(pop_pool)

# the fitness function evaluates how well a gene does. in this case we insert the weights
# into a player into npare games, which i've changed it to return the win percentage
# pebs_generator = lambda name,w:JogadorLinearPond(name,DEPTH,w,HEURISTICS,100)
# fitness= lambda name, w:jogaNpares_GA(jogo,N_GAMES,TRAINING_ENEMY,pebs_generator(name,w),1)

def fitness (tuple): #tuple(name:int, weights:list[float], fitness:float)
    name = tuple[0]
    weights = tuple[1]
    pebs_generator = lambda name,w:JogadorLinearPond(name,DEPTH,w,HEURISTICS,100)
    return (name,weights,jogaNpares_GA(jogo,N_GAMES,TRAINING_ENEMY,pebs_generator(tuple[0],tuple[1]),1))

#call the genetic algorithm
best_fit_weights = ga_kev(pop,fitness,[],ngen=N_GENERATION,pmut=PMUT)
#pop_pool ignored

print(best_fit_weights)

[[0.5408094183539636, 0.00284445612406821, 0.15969595598418704, 0.22260996169213218, 0.07404020784564898], [0.5060064283744496, 0.0023946325203097678, 0.27945500045902055, 0.17827186795515407, 0.03387207069106592], [0.5087305384943449, 0.0030431687499736727, 0.17085225489545827, 0.2381614091784416, 0.07921262868178149], [0.5492302031235177, 0.002888746315759254, 0.15673364697615585, 0.2184806179605766, 0.07266678562399057], [0.41752397451100454, 0.0012991470449279108, 0.29854278707734244, 0.21209212302177424, 0.07054196834495088], [0.4293397173643812, 0.0012727933638494399, 0.2924867355859959, 0.20778975540973707, 0.06911099827603649], [0.43822936301080534, 0.001363572912484629, 0.31334779174201893, 0.20761242699559582, 0.0394468453390953], [0.5141678293348594, 0.002536050034560504, 0.16912696677170333, 0.2357564245263881, 0.07841272933248868], [0.5115214949486129, 0.0011768297898419075, 0.23384833697495405, 0.19019452446170035, 0.06325881382489079], [0.5244964022687261, 0.002482134754

overall gen:   0%|          | 0/1 [00:00<?, ?it/s]

jogaNpares_GA range 1:   0%|          | 0/50 [00:00<?, ?it/s]

eval pop:   0%|          | 0/24 [00:00<?, ?it/s]

jogaNpares_GA range 5:  94%|█████████▍| 47/50 [00:35<00:02,  1.26it/s]

jogaNpares_GA range 6:  94%|█████████▍| 47/50 [00:35<00:02,  1.18it/s]

In [ ]:
pop_gen17 = [(296, [0.5408094183539636, 0.00284445612406821, 0.15969595598418704, 0.22260996169213218, 0.07404020784564898], 0.5922330097087378), (181, [0.5060064283744496, 0.0023946325203097678, 0.27945500045902055, 0.17827186795515407, 0.03387207069106592], 0.5873015873015873), (326, [0.5087305384943449, 0.0030431687499736727, 0.17085225489545827, 0.2381614091784416, 0.07921262868178149], 0.5849056603773585), (281, [0.5492302031235177, 0.002888746315759254, 0.15673364697615585, 0.2184806179605766, 0.07266678562399057], 0.5754716981132075), (247, [0.41752397451100454, 0.0012991470449279108, 0.29854278707734244, 0.21209212302177424, 0.07054196834495088], 0.573208722741433), (340, [0.4293397173643812, 0.0012727933638494399, 0.2924867355859959, 0.20778975540973707, 0.06911099827603649], 0.5728155339805825), (218, [0.43822936301080534, 0.001363572912484629, 0.31334779174201893, 0.20761242699559582, 0.0394468453390953], 0.5705128205128205), (328, [0.5141678293348594, 0.002536050034560504, 0.16912696677170333, 0.2357564245263881, 0.07841272933248868], 0.5682539682539682), (304, [0.5115214949486129, 0.0011768297898419075, 0.23384833697495405, 0.19019452446170035, 0.06325881382489079], 0.565359477124183), (216, [0.5244964022687261, 0.0024821347540840543, 0.16553140370103708, 0.23074434922059944, 0.07674571005555327], 0.565359477124183), (183, [0.5408094183539636, 0.00284445612406821, 0.15969595598418704, 0.22260996169213218, 0.07404020784564898], 0.565359477124183), (191, [0.4220340143246191, 0.0016217516845741953, 0.3222589517171829, 0.21351662545668781, 0.040568656816935986], 0.5638629283489096), (361, [0.5321287927976377, 0.0024422935796346032, 0.1628744304964742, 0.22704063174297384, 0.0755138513832796], 0.5631067961165048), (244, [0.503236853071467, 0.003219031462752779, 0.2098999632357117, 0.23835596329214934, 0.04528818893791916], 0.5566037735849056), (343, [0.5278617454660057, 0.0012144229189132074, 0.16479734850271316, 0.22972110477729635, 0.07640537833507148], 0.5533980582524272), (286, [0.47551148641798996, 0.0012635841717298889, 0.2510873362804695, 0.204215420729435, 0.06792217240037564], 0.5533980582524272), (337, [0.4824081948782528, 0.00253728743925837, 0.2879892244193175, 0.19081079683976004, 0.036254496423411206], 0.5512820512820513), (119, [0.4567526386179326, 0.0014511037277271798, 0.28834942524079316, 0.04219666311629308, 0.21125016929725385], 0.5471698113207547), (321, [0.4240589846181617, 0.0016295330454913263, 0.3238051891812408, 0.21050907747006667, 0.03999721568503944], 0.545751633986928), (268, [0.4589515295334259, 0.002413914115632853, 0.24539703161033458, 0.2200490707654506, 0.07318845397515614], 0.545751633986928), (145, [0.42096729007497213, 0.0016247448742976692, 0.3228537296919815, 0.21391070290713027, 0.04064353245161829], 0.545751633986928), (205, [0.5242839050426428, 0.0027575380787338167, 0.17980785464343735, 0.21998391808195197, 0.07316678415323409], 0.5451713395638629), (305, [0.4641343575158191, 0.0012333515476795063, 0.24507979898187618, 0.21728377651802178, 0.07226871543660336], 0.5416666666666666), (239, [0.5408094183539636, 0.00284445612406821, 0.15969595598418704, 0.22260996169213218, 0.07404020784564898], 0.5416666666666666)]
pop = []
for ind in pop_gen17:
    pop.append(ind[1])
# pop_pool = generate_pool(PRECISION)
print(pop)

[[0.5408094183539636, 0.00284445612406821, 0.15969595598418704, 0.22260996169213218, 0.07404020784564898], [0.5060064283744496, 0.0023946325203097678, 0.27945500045902055, 0.17827186795515407, 0.03387207069106592], [0.5087305384943449, 0.0030431687499736727, 0.17085225489545827, 0.2381614091784416, 0.07921262868178149], [0.5492302031235177, 0.002888746315759254, 0.15673364697615585, 0.2184806179605766, 0.07266678562399057], [0.41752397451100454, 0.0012991470449279108, 0.29854278707734244, 0.21209212302177424, 0.07054196834495088], [0.4293397173643812, 0.0012727933638494399, 0.2924867355859959, 0.20778975540973707, 0.06911099827603649], [0.43822936301080534, 0.001363572912484629, 0.31334779174201893, 0.20761242699559582, 0.0394468453390953], [0.5141678293348594, 0.002536050034560504, 0.16912696677170333, 0.2357564245263881, 0.07841272933248868], [0.5115214949486129, 0.0011768297898419075, 0.23384833697495405, 0.19019452446170035, 0.06325881382489079], [0.5244964022687261, 0.002482134754

In [ ]:
list1 = [[1, 2, 3, 4, 5],0.1]
list2 = [['a', 'b', 'c', 'd', 'e'],0.4]
list3 = [[0, 9, 8, 7, 6],0.2]

def wah(*lists):
    listee = [x[0] for x in [*lists]]
    print(listee)
    return [list(value) for value in zip(*listee)]
babha = wah(list1,list2,list3)
print(babha)


listx = []
listx.append(list1)
listx.append(list2)
print(listx)

[[1, 2, 3, 4, 5], ['a', 'b', 'c', 'd', 'e'], [0, 9, 8, 7, 6]]
[[1, 'a', 0], [2, 'b', 9], [3, 'c', 8], [4, 'd', 7], [5, 'e', 6]]
[[[1, 2, 3, 4, 5], 0.1], [['a', 'b', 'c', 'd', 'e'], 0.4]]



# Tornament

In [ ]:
def incorpora(tabela,tx):
    for jog in tx:
        if jog not in tabela:
            tabela[jog]=tx[jog]
        else:
            tabela[jog]+=tx[jog]
    
def torneio(n,jogadores,jogo_id=0):
    jogo=Kalah(jogo_id)  # jogo gerado ao calha entre os "justos"
    tabela={}
    for i in range(len(jogadores)-1):
        jog1=jogadores[i]
        for j in range(i+1,len(jogadores)):
            jog2=jogadores[j]
            _,_,_,tabelaX = jogaNpares(jogo,n,jog1,jog2)
            incorpora(tabela,tabelaX)
    return tabela

In [ ]:
# # from tqdm import tqdm
# from random import sample
# # for i in tqdm(range(10000000)):
# #     pass

# x = sorted(sample(range(1, 100), 5 - 1))
# print(x)
jogs = [toe1,pebs,pebs2,pebs4,pebs2b,pebs4b]
# jogs = [toe1,pebs,pebs2,pebs2b,pebs3b,pebs4b]
torneio(100,jogs,0)

{'chapiteau': 39,
 'Pebbles': 1757,
 'Pebbles2-defense-focused': 1757,
 'Pebbles4-defense-focused': 1786,
 'Pebbles2-capture-focused': 1750,
 'Pebbles4-capture-focused': 1763}

In [ ]:

'''
      _   _  _  _  _  _
   _  12 11 10  9  8  7     N
   13                    6 
       0  1  2  3  4  5     S

'''
board = [5, 14, 6, 14, 14, 0, 0, 14, 1, 0, 14, 14, 14, 0]

south_max_capture = 0
south_empties = [i for i in range(6) if board[i]==0]
south_move_final_index = [(i+board[i])%14 if i+board[i]<13 else (i+board[i])%13 for i in range(6) if board[i] != 0 and board[i]<=13]
south_possible_captures = [value for value in south_empties if value in south_move_final_index]

if len(south_possible_captures) != 0:
    for i in south_possible_captures:
        if 1+board[i+7] > south_max_capture:
            south_max_capture = 1+board[i+7]

north_max_capture = 0
north_empties = [i for i in range(7,13) if board[i]==0]
north_move_final_index = [(i+board[i])%14 if i+board[i]-7<13 else ((i+board[i])%14)+1 for i in range(7,13) if board[i] != 0 and board[i]<=13]
north_possible_captures = [value for value in north_empties if value in north_move_final_index]

if len(north_possible_captures) != 0:
    for i in north_possible_captures:
        print("1+board[i-7]",1+board[i-7], "i",i)
        if 1+board[i-7] > north_max_capture:
            north_max_capture = 1+board[i-7]


# north_move_final_index = [(i+board[i])%14 if i+board[i]-7<13 else (i+(board[i]%13))%13 for i in range(7,13)]
print(south_empties)
print(south_move_final_index)
print(south_possible_captures)
print(south_max_capture)

print(north_empties)
print(north_move_final_index)
print(north_possible_captures)
print(north_max_capture)




#0,1,2,3,4,6

1+board[i-7] 7 i 9
[5]
[5, 8]
[5]
15
[9]
[9]
[9]
7
